# Finding linguistic patterns using spaCy

*Content adapated from [Tuomo Hiipala'](https://www.mv.helsinki.fi/home/thiippal/) lectures

This section teaches you to find linguistic patterns using spaCy, a natural language processing library for Python.

After this section, you should:

 - know how to search for patterns based on part-of-speech tags and morphological features
 - know how to search for patterns based on syntactic dependencies

Linguistic annotations, such as part-of-speech tags, syntactic dependencies and morphological features, help impose structure on written language. Crucially, linguistic annotations allow searching for structural patterns instead of individual words or phrases. This allows defining search patterns in a flexible way.

In the spaCy library, the capability for pattern search is provided by various components named *Matchers*.

spaCy provides three types of *Matchers*:

1. A [Matcher](https://spacy.io/api/matcher), which allows defining rules that search for particular **words or phrases** by examining *Token* attributes.  
2. A [DependencyMatcher](https://spacy.io/api/dependencymatcher), which allows searching parse trees for **syntactic patterns**.
3. A [PhraseMatcher](https://spacy.io/api/phrasematcher), a fast method for matching spaCy *Doc* objects to *Doc* objects.

### Matching words or phrases

In [ ]:
# importing the spaCy library into Python
import spacy

# loading a small language model for English; assign the result under 'nlp'
nlp = spacy.load('en_core_web_sm')

In [ ]:
# let's again load our text
with open('data/treaty_of_lisbon.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

# and feed it to spacy
doc = nlp(text)

In [ ]:
# just checking the length of our object
print(len(doc))

In [ ]:
# importing the Matcher class
from spacy.matcher import Matcher

Importing the *Matcher* class from spaCy's `matcher` submodule allows creating *Matcher* objects.

When creating a *Matcher* object, you must provide the vocabulary of the language model used for finding matches to the *Matcher* object.

The reason for this is really rather simple: if you want to search for patterns in some language, you need to know its vocabulary first.

spaCy stores the vocabulary of a model in a [*Vocab*](https://spacy.io/api/vocab) object. The *Vocab* object can be found under the attribute `vocab` of a spaCy *Language* object.

In this case, we have the *Language* object that contains a small language model for English stored under the variable `nlp`, which means we can access its *Vocab* object under the attribute `nlp.vocab`.

We then call the *Matcher* **class** and provide the vocabulary under `nlp.vocab` to the `vocab` argument to create a *Matcher* object. We store the resulting object under the variable `matcher`.

In [ ]:
# creating a Matcher and providing model vocabulary
matcher = Matcher(vocab=nlp.vocab)

# checking it out
matcher

The *Matcher* object is now ready to store the patterns that we want to search for.

These patterns, or more specifically, *pattern rules*, are created using a [specific format](https://spacy.io/api/matcher#patterns) defined in spaCy.

Each pattern consists of a Python list, which is populated by Python dictionaries. 

Each dictionary in this list describes the pattern for matching a single spaCy *Token*. 

If you wish to match a sequence of *Tokens*, you must define multiple dictionaries within a single list, whose order follows that of the pattern to be matched.

Let's start by defining a simple pattern for matching sequences of pronouns and verbs, and store this pattern under the variable `pronoun_verb`.

This pattern consists of a list, as marked by the surrounding brackets `[]`, which contains two dictionaries, marked by curly braces `{}` and separated by a comma. The key and value pairs in a dictionary are separated by a colon.

 - The dictionary key determines which *Token* attribute should be searched for matches. The attributes supported by the *Matcher* can be found [here](https://spacy.io/api/matcher#patterns).

 - The value under the dictionary key determines the specific value for the attribute.

In this case, we define a pattern that searches for a sequence of two coarse part-of-speech tags (`POS`), namely pronouns (`PRON`) and verbs (`VERB`).

Note that both keys and values must be provided in uppercase letters.

In [ ]:
# defining a list with nested dictionaries that contains the pattern to be matched
pronoun_verb = [{'POS': 'PRON'}, {'POS': 'VERB'}]

Now that we have defined the pattern using a list and dictionaries, we can add it to the *Matcher* object under the variable `matcher`.

This can be achieved using `add()` method, which requires two inputs:

 1. A Python string object that defines a name for the pattern. This is required for purposes of identification.
 2. A list containing the pattern(s) to be searched for. Because a single rule for matching patterns can contain multiple patterns, the input must be a *list of lists*. We therefore wrap the input lists into brackets, e.g. `[pattern_1]`.

In [ ]:
# adding the pattern to the matcher under the name 'pronoun+verb'
matcher.add("pronoun+verb", patterns=[pronoun_verb])

To search for matches in the *Doc* object stored under the variable `doc`, we feed the *Doc* object to the *Matcher* and store the result under the variable `result`.

We also set the optional argument `as_spans` to `True`, which instructs spaCy to return the results as *Span* objects. *Span* objects correspond to continuous "slices" (sequences of *tokens*) of *Doc* objects.

In [ ]:
# applying the Matcher to the Doc object under 'doc' with the argument
# 'as_spans' set to True to get Spans as output
result = matcher(doc, as_spans=True)

# calling the variable to examine the output
result

The output is a list of spaCy *Span* objects that match the requested pattern. Let's examine the first object in the list of matches in greater detail.

In [ ]:
result[0]

The *Span* object has various useful [attributes](https://spacy.io/api/span), including `start` and `end`. These attributes contain the indices that indicate where in the *Doc* object the *Span* starts and finishes.

In [ ]:
result[0].start, result[0].end

In [ ]:
result[0].label

In [ ]:
result[0].label_

In [ ]:
print(result[0].start, result[0].end)
print(result[0].label)
print(result[0].label_)

The number stored under the `label` attribute is actually a spaCy [*Lexeme*](https://spacy.io/api/lexeme) object that corresponds to an entry in the language model's vocabulary. 

This *Lexeme* contains the name that we gave to the search pattern above, namely `pronoun+verb`.

Another way to verify this by using the value under `result[0].label` to fetch the *Lexeme* from the *Vocab* object under `nlp.vocab` and examining its `text` attribute.

In [ ]:
nlp.vocab[result[0].label].text

The information under the `label` attribute is useful for disambiguating between patterns, especially if the same *Matcher* object contains multiple different patterns, as we will see shortly below.

Looking at the matches above, the pattern we defined is quite restrictive, as the pronoun and the verb must follow each other.

We cannot, for example, match patterns in which the verb is preceded by auxiliary verbs.

spaCy allows increasing the flexibility of pattern rules using operators (similar to operators in regular expressions). 

These operators are defined by adding the key `OP` to the dictionary that defines a pattern for a single *Token*. spaCy supports the following operators:

 - `!`: Negate the pattern; the pattern can occur exactly zero times.
 - `?`: Make the pattern optional; the pattern may occur zero or one times.
 - `+`: Require the pattern to occur one or more times.
 - `*`: Allow the pattern to match zero or more times.

Let's explore the use of operators by defining another pattern rule, which extends the scope of our *Matcher*.

To do so, we define another pattern for a *Token* between the pronoun and the verb. This *Token* must have the coarse part-of-speech tag `AUX`, which indicates an auxiliary verb:

```python
{'POS': 'AUX', 'OP': '+'}
```

In addition, we add another key and value pair to the dictionary for this *Token*, which contains the key `OP` with the value `+`. This means that the *Token* corresponding to an auxiliary verb must occur *one or more times*.

We store the resulting list with nested dictionaries under the variable `pronoun_aux_verb`, and add the pattern to the existing *Matcher* object stored under the variable `matcher`.

In [ ]:
# defining a list with nested dictionaries that contains the pattern to be matched
pronoun_aux_verb = [{'POS': 'PRON'}, {'POS': 'AUX', 'OP': '+'}, {'POS': 'VERB'}]

# adding the pattern to the matcher under the name 'pronoun+aux+verb'
matcher.add('pronoun+aux+verb', patterns=[pronoun_aux_verb])

results = matcher(doc, as_spans=True)

In [ ]:
results

### Matching morphological features

The `morph` attribute contains a string object, in which each morphological feature is separated by a vertical bar `|`, as illustrated below.

```
We 	 Case=Nom|Number=Plur|Person=1|PronType=Prs
```

As you can see, particular types of morphological features, e.g. *Case*, and their type, e.g. *Nom* (for the nominative case) are separated by equal signs `=`.

Let's begin exploring how we can define pattern rules that match morphological features.

To get started, we create a new *Matcher* object named `morph_matcher`.

In [ ]:
# creating a Matcher and provide model vocabulary
morph_matcher = Matcher(vocab=nlp.vocab)

We then define a new pattern with rules for two *Tokens*:

 1. Tokens that have a fine-grained part-of-speech tag `NNP` (proper noun), which can occur one or more times (operator: `+`).
 
 ```python
 {'TAG': 'NNP', 'OP': '+'}
 ```
 
 
 2. Tokens that have a coarse part-of-speech tag `VERB` and have *all* the following morphological features (`MORPH`): `Number=Sing|Person=Three|Tense=Pres|VerbForm=Fin`.
 
 ```python
 {'POS': 'VERB', 'MORPH': 'Number=Sing|Person=Three|Tense=Pres|VerbForm=Fin'}
 ```
 
We define the pattern using two dictionaries in a list, which we assign under the variable `propn_3rd_finite`.

In [ ]:
propn_3rd_finite = [{'TAG': 'NNP', 'OP': '+'},
                    {'POS': 'VERB', 'MORPH': 'Number=Sing|Person=Three|Tense=Pres|VerbForm=Fin'}]

We then add the pattern to the newly-created *Matcher* stored under the variable `morph_matcher` using the `add()` method.

We also provide the value `LONGEST` to the optional argument `greedy` for the `add()` method.

The `greedy` argument filters the matches for *Tokens* that include operators such as `+` that search *greedily* for more than one match.

By setting the value to `LONGEST`, spaCy returns the longest sequence of matches instead of returning a match every time it finds one. Put differently, spaCy will collect all the matching *Tokens* before returning them.

In [ ]:
# adding the pattern to the matcher under the name 'sing_3rd_pres_fin'
morph_matcher.add('sing_3rd_pres_fin', patterns=[propn_3rd_finite], greedy='LONGEST')

In [ ]:
# then again
morph_results = morph_matcher(doc, as_spans=True)

In [ ]:
morph_results

As you can see, the matches are relatively few in number, because we defined that the verb should have quite specific morphological features.

The question is, then, how can we match just *some* morphological features?

To loosen the criteria for morphological features by focusing on [tense](https://en.wikipedia.org/wiki/Grammatical_tense) only, we need to use a dictionary with the key `MORPH`, but instead of a string object, we provide a dictionary as its value:

For this dictionary, we use the string `IS_SUPERSET` as the key. `IS_SUPERSET` is one of the attributes defined in the spaCy [pattern format](https://spacy.io/api/matcher#patterns), e.g.

```python
{'MORPH': {'IS_SUPERSET': [...]}}
```

Before proceeding any further, let's unpack the logic behind `IS_SUPERSET` a bit.

We can think of morphological features associated with a given Token as a [set](https://en.wikipedia.org/wiki/Set_(mathematics)). To exemplify, a set could consist of the following four items:

```
Number=Sing, Person=Three, Tense=Pres, VerbForm=Fin
```

If we would have *another set* with just one item, `Tense=Pres`, we could describe the relationship between the two sets by stating that the first set (with four items) is a superset of the second set (with one item).

In other words, the larger (super)set contains the smaller (sub)set.

This is also how matching using `IS_SUPERSET` works: spaCy retrieves the morphological features for each *Token*, and examines whether these features are a superset of the features defined in the search pattern.

The morphological features to be searched for are provided as a list of Python strings.

These strings, in turn, define particular morphological features, e.g. `Tense=Past`, as defined in the [Universal Dependencies](https://universaldependencies.org/u/overview/morphology.html) schema for describing morphology.

This list is then used as the value for the key `IS_SUPERSET`.

Let's now proceed to search for verbs in the past tense and add them to the *Matcher* object under `morph_matcher`.

In [ ]:
# define a list with nested dictionaries that contains the pattern to be matched
past_tense = [{'TAG': 'NNP', 'OP': '+'},
              {'POS': 'VERB', 'MORPH': {'IS_SUPERSET': ['Tense=Past']}}]

# adding the pattern to the matcher under the name 'past_tense'
morph_matcher.add('past_tense', patterns=[past_tense], greedy='LONGEST')

morph_results = morph_matcher(doc, as_spans=True)

In [ ]:
# looping over each Span object in the list 'results'
for result in morph_results:
    
    # printing out the the name of the pattern rule, the matching Span, and the morphological features of the
    # last Token in the match (a verb).
    print(nlp.vocab[result.label].text, '\t', result, '\t', result[-1].morph)

### Matching syntactic dependencies

If you want to match patterns based on syntactic dependencies, you must use the *DependencyMatcher* class in spaCy.

As we learned, syntactic dependencies describe the relations that hold between *Token* objects.

To get started, let's import the *DependencyMatcher* class from the `matcher` submodule.

As you can see, the *DependencyMatcher* is initialised just as like the *Matcher* above.

In [ ]:
# importing the DependencyMatcher class
from spacy.matcher import DependencyMatcher

# creating a DependencyMatcher and provide model vocabulary
dep_matcher = DependencyMatcher(vocab=nlp.vocab)

This provides us with a *DependencyMatcher* stored under the variable `dep_matcher`, which is now ready for storing dependency patterns.

When developing pattern rules for matching syntactic dependencies, the first step is to determine an "anchor" around which the pattern is built.

Visualising the syntactic dependencies, can help formulate patterns. 

Let's import the displacy submodule to draw the syntactic dependencies for a sentence in the *Doc* object stored under the variable `doc`.

In [ ]:
# importing the displacy submodule from spaCy
from spacy import displacy

# let's try with sentence in postion 45
displacy.render(list(doc.sents)[45], style='dep')

Syntactic dependencies are visualised using arcs that lead from the *head* *Token* to the *dependent* *Token*. The label of the arc gives the syntactic dependency.

To exemplify, the verb "remains" governs the noun "security". "security", in turn governs the adjective "national".

Let's define a pattern that searches for verbs and their nominal subjects (`nsubj`).

Just as using the *Matcher* class, the pattern rules for the *DependencyMatcher* are defined using a list of dictionaries.

The first dictionary in the list defines an "anchor" pattern and its attributes.

You can think of the pattern rule as a chain that proceeds from left to right, and the first pattern on the left acts as an anchor for the subsequent patterns on its right-hand side.

Hence we define the following pattern for the anchor:

```python
{'RIGHT_ID': 'verb', 'RIGHT_ATTRS': {'POS': 'VERB'}}
```

We use the required key `RIGHT_ID` to provide a name for this pattern, which can be then used to refer to this pattern by subsequent patterns on its **right-hand side**.

In other words, when you see the key `RIGHT_ID`, think of a name for the *current pattern*.

We then create a dictionary under the key `RIGHT_ATTRS` that holds the linguistic features of the anchor. In this case, we determine that the anchor should have `VERB` as its part-of-speech tag.

Next, we determine a pattern for the next "link" in the chain to the right of the anchor:

```python
{'LEFT_ID': 'verb', 'REL_OP': '>', 'RIGHT_ID': 'subject', 'RIGHT_ATTRS': {'DEP': 'nsubj'}}
```

We start by providing the key `LEFT_ID`, whose value is a string object that refers to the name of a pattern on the **left-hand side** of the current pattern. This is the name that we gave to the anchor using the key `RIGHT_ID`.

Next, we use the key `REL_OP` to define a [relation operator](https://spacy.io/api/dependencymatcher#patterns), which determines the relationship between this pattern and that referred to using `LEFT_ID`.

The relation operator `>` defines that the pattern under `LEFT_ID` – the anchor – should be the head of the current pattern.

Next, we name the current pattern using the key `RIGHT_ID`, which enables referring to this pattern on the right-hand side, if necessary. We give this pattern the name `subject`.

We then use the `RIGHT_ATTRS` to determine the attributes for the current pattern. We define that the syntactic relation that holds between this pattern and that on the left should be `nsubj` or nominal subject.

In [ ]:
# let's use a smaller piece of text...
with open('data/treaty_of_lisbon.txt', 'r', encoding='UTF-8') as f:
    text = f.read()
    
ind_begin = text.find('Article 8 A')
ind_end = text.find('Article 8 B')

text = text[ind_begin:ind_end]

doc = nlp(text)

In [ ]:
# defining a list with nested dictionaries that contains the pattern to be matched
dep_pattern = [{'RIGHT_ID': 'verb', 'RIGHT_ATTRS': {'POS': 'VERB'}},
               {'LEFT_ID': 'verb', 'REL_OP': '>', 'RIGHT_ID': 'subject', 'RIGHT_ATTRS': {'DEP': 'nsubj'}}
              ]

In [ ]:
# adding the pattern to the matcher under the name 'nsubj_verb'
dep_matcher.add('nsubj_verb', patterns=[dep_pattern])

# applying the DependencyMatcher to the Doc object under 'doc'
dep_matches = dep_matcher(doc)

# checking it out
dep_matches

Unlike the *Matcher*, the *DependencyMatcher* cannot return the matches as *Span* objects, because the matches do not necessarily form a continuous sequence of *Tokens* needed for a *Span* object.

Thus the *DependencyMatcher* returns a list of tuples. 

Each tuple contains two items:

 1. A *Lexeme* object that gives the name of the pattern
 2. A list of indices for *Tokens* that match the search pattern in the *Doc* object

In [ ]:
# looping over each tuple in the list 'dep_matches'
for match in dep_matches:
    
    # taking the first item in the tuple at [0] and assigning it under
    # the variable 'pattern_name'. This item is a spaCy Lexeme object.
    pattern_name = match[0]
    
    # taking the second item in the tuple at [1] and assigning it under
    # the variable 'matches'. This is a list of indices referring to the
    # Doc object under 'doc' that we just matched.
    matches = match[1]
    
    # let's unpack the matches list into variables for clarity
    verb, subject = matches[0], matches[1]
    
    # printing the matches by first fetching the name of the pattern from the 
    # Vocabulary object. Next, use the 'subject' and 'verb' variables to 
    # index the Doc object. This gives us the actual Tokens matched.
    print(nlp.vocab[pattern_name].text, '\t', doc[subject], '...', doc[verb])

### Exercise

1. Find the direct objects (dobj) for the verbs matched above. Hint: you should **not** add this as a link to the existing chain whose rightmost item is currently named `subject`. Instead, you need to start a new chain that begins from the anchor pattern `verb`.

### Solution

In [ ]:
# defining a list with nested dictionaries that contains the pattern to be matched
dep_pattern_2 = [{'RIGHT_ID': 'verb', 'RIGHT_ATTRS': {'POS': 'VERB'}},
                 {'LEFT_ID': 'verb', 'REL_OP': '>', 'RIGHT_ID': 'subject', 'RIGHT_ATTRS': {'DEP': 'nsubj'}},
                 {'LEFT_ID': 'verb', 'REL_OP': '>', 'RIGHT_ID': 'd_object', 'RIGHT_ATTRS': {'DEP': 'dobj'}}
                ]

# adding the pattern to the matcher under the name 'nsubj_verb'
dep_matcher.add('nsubj_verb_dobj', patterns=[dep_pattern_2])

# applying the DependencyMatcher to the Doc object under 'doc'
dep_matches = dep_matcher(doc)

# looping over each tuple in the list 'dep_matches'
for match in dep_matches:
    
    # taking the first item in the tuple at [0] and assigning it under
    # the variable 'pattern_name'. This item is a spaCy Lexeme object.
    pattern_name = match[0]
    
    # taking the second item in the tuple at [1] and assigning it under
    # the variable 'matches'. This is a list of indices referring to the
    # Doc object under 'doc' that we just matched.
    matches = match[1]

    # because we now have two patterns for matching which return lists of
    # different length, e.g. lists with two indices for 'nsubj_verb' and
    # lists with three indices for 'nsubj_verb_dobj', we must now define
    # conditional criteria for handling these lists.
    if len(matches) > 2:
        
        # let's unpack the matches list into variables for clarity
        verb, subject, dobject = matches[0], matches[1], matches[2]
    
        # printing the matches by first fetching the name of the pattern from the 
        # Vocabulary object. Next, use the 'subject' and 'verb' variables to 
        # index the Doc object. This gives us the actual Tokens matched.
        print(nlp.vocab[pattern_name].text, '\t', doc[subject], '...', doc[verb], '...', doc[dobject])
        
    # alternative condition with just two items in the list.
    else:
        
        # let's unpack the matches list into variables for clarity
        verb, subject = matches[0], matches[1]
    
        print(nlp.vocab[pattern_name].text, '\t', doc[subject], '...', doc[verb])